In [1]:
import pandas as pd
import numpy as np
from glob import glob

In [2]:
# global variables

char = "\\"
columns = ["date", "tavg", "tmin", "tmax", "prcp"]
t_columns = [i+"_x" for i in columns]
s_columns = [i+"_y" for i in columns]
op_into = False
empty_pred_target = "GqIUVenONyZikTIz"

In [3]:
# get pathsfor daily and target

min_daily_weather_paths = [f for f in glob("../data/weather/minnesota_daily/*.csv")]
pred_targets_paths = [f for f in glob("../data/weather/prediction_targets_daily/*.csv") if empty_pred_target not in f]

print(len(min_daily_weather_paths))
print(len(pred_targets_paths))

97
197


In [4]:
# read in all daily weather csvs

min_daily_weather_dict = {}

for path in min_daily_weather_paths:

    station_code = path.split(char)[-1].split(".")[0]

    min_daily_weather_dict[station_code] = pd.read_csv(filepath_or_buffer=path, names=columns, index_col="date")
    
    if op_into:
        print(f"{station_code}:\n\tfrom: {min_daily_weather_dict[station_code].index[0]}\n\tto: {min_daily_weather_dict[station_code].index[-1]}")

In [5]:
# calculate weights for averaging features

# all_stations_merged = pd.concat((pd.read_csv(filepath_or_buffer=path) for path in min_daily_weather_paths), ignore_index=True)
# all_pred_targets_merged = pd.concat((pd.read_csv(filepath_or_buffer=path) for path in pred_targets_paths), ignore_index=True)
df_úristen_verybig = pd.concat((pd.concat((pd.read_csv(filepath_or_buffer=path) for path in min_daily_weather_paths), ignore_index=True), pd.concat((pd.read_csv(filepath_or_buffer=path) for path in pred_targets_paths), ignore_index=True)), ignore_index=True)

num_nans = df_úristen_verybig.isna().sum()

del df_úristen_verybig

In [6]:
# read sequentally all prediction targets and compare with all station data

distances = {"Prediction Target": [],
             "Staion": [],
             "L2 distance": []}

for i, path in enumerate(pred_targets_paths):

    prediction_code = path.split(char)[-1].split(".")[0]

    df_target = pd.read_csv(filepath_or_buffer=path, names=columns, index_col="date")

    for station in min_daily_weather_dict.keys():
        
        # 1, check date correspondence
        tmp = pd.merge(left=df_target, right=min_daily_weather_dict[station], left_index=True, right_index=True, how="inner")

        if op_into:
            print(tmp)

        # 2, calculate average L2 distance between all measured modalities
        dist_t_avg = (tmp["tavg_x"] - tmp["tavg_y"])**2
        dist_t_min = (tmp["tmin_x"] - tmp["tmin_y"])**2
        dist_t_max = (tmp["tmax_x"] - tmp["tmax_y"])**2
        dist_t_prcp = (tmp["prcp_x"] - tmp["prcp_y"])**2

        distance = np.nanmean([dist_t_avg, dist_t_min, dist_t_max, dist_t_prcp])

        distances["Prediction Target"].append(prediction_code)
        distances["Staion"].append(station)
        distances["L2 distance"].append(distance)

df_distances = pd.DataFrame(data=distances)

C:\Users\ktama\AppData\Local\Temp\ipykernel_3684\2895047263.py:27: RuntimeWarning: Mean of empty slice
  distance = np.nanmean([dist_t_avg, dist_t_min, dist_t_max, dist_t_prcp])
C:\Users\ktama\AppData\Local\Temp\ipykernel_3684\2895047263.py:27: RuntimeWarning: Mean of empty slice
  distance = np.nanmean([dist_t_avg, dist_t_min, dist_t_max, dist_t_prcp])
C:\Users\ktama\AppData\Local\Temp\ipykernel_3684\2895047263.py:27: RuntimeWarning: Mean of empty slice
  distance = np.nanmean([dist_t_avg, dist_t_min, dist_t_max, dist_t_prcp])
C:\Users\ktama\AppData\Local\Temp\ipykernel_3684\2895047263.py:27: RuntimeWarning: Mean of empty slice
  distance = np.nanmean([dist_t_avg, dist_t_min, dist_t_max, dist_t_prcp])
C:\Users\ktama\AppData\Local\Temp\ipykernel_3684\2895047263.py:27: RuntimeWarning: Mean of empty slice
  distance = np.nanmean([dist_t_avg, dist_t_min, dist_t_max, dist_t_prcp])
C:\Users\ktama\AppData\Local\Temp\ipykernel_3684\2895047263.py:27: RuntimeWarning: Mean of empty slice
  dista

In [7]:
df_distances

,Prediction Target,Staion,L2 distance
0,ACAvNTuEuFWcmwms,72644,15.833362
1,ACAvNTuEuFWcmwms,72655,10.076447
2,ACAvNTuEuFWcmwms,72658,17.110965
3,ACAvNTuEuFWcmwms,72745,9.552607
4,ACAvNTuEuFWcmwms,72747,19.582658
...,...,...,...
19104,zuWktVTnYICcsVFr,KY630,53.205526
19105,zuWktVTnYICcsVFr,P6529,76.529128
19106,zuWktVTnYICcsVFr,UYB6K,89.585664
19107,zuWktVTnYICcsVFr,X9FED,73.584732


In [8]:
df_distances.groupby(['Prediction Target'])['L2 weighted distance'].min()

KeyError: 'Column not found: L2 weighted distance'